# 스파크 스트리밍 실습 2교시 : 스파크 스트리밍 애플리케이션

> 정적인 데이터인 파일 스트림외에도 다양한 소스를 통해 스트림 처리를 할 수 있는데, 이번 장에서는 소켓을 통해 직접 스트림을 생성하고, 이를 처리하는 애플리케이션을 구현하여 보다 동적인 실습을 합니다. 그리고 1교시에서 학습했던 5가지 스트리밍 애플리케이션 단계를 반복 숙지하고 좀 더 깊이 이해합니다 `Spark > Reader > Transform > Writer > Trigger`

## 학습 목표
* 요구사항을 분석하고 스트리밍 애플리케이션을 설계하기
* 소켓 스트림 리더를 생성하고, 워드 카운터 변환코드를 작성합니다
* 콘솔 출력 싱크 및 트리거를 생성하고, 넷캣(nc)을 통해 소켓 서버를 기동합니다
* 스트리밍 애플리케이션을 기동하고, 소켓서버에서 생성되는 스트리밍을 확인합니다
* 워드 카운트 결과를 확인하고 애플리케이션을 종료합니다

## 목차
* [1. 소켓 스트림 리더 생성](#1.-소켓-스트림-리더-생성)
* [2. 워드 카운트 변환 로직 생성](#2.-워드-카운트-변환-로직-생성)
* [3. 콘솔 출력 생성](#3.-콘솔-출력-생성)
* [4. 트리거 생성](#4.-트리거-생성)
* [5. 애플리케이션 기동 및 소켓 출력](#5.-애플리케이션-기동-및-소켓-출력)
* [6. 스파크 스트리밍 실행과정 이해](#6.-스파크-스트리밍-실행과정-이해)


In [3]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

<br>

## 1. 소켓 스트림 리더 생성

#### Step 1: 입력 소스 테이블 정의하기 - { spark, reader }

* 스파크 세션을 생성하는 과정을 동일하며, spark.read 대신 readStream 을 통해 데이터프레임을 생성하는 점이 다릅니다
  - DataFrameReader 와 유사한 DataStreamReader 를 통해 데이터를 읽어오게 됩니다
* 여기서 lines 는 localhost:9999 소켓으로부터 여러줄의 텍스트 데이터를 가진 unbounded table 을 정의합니다 (not running but declare)
  - DataFrameReader 는 text 뿐만 아니라 Parquet, ORC, JSON 등 다양한 포맷까지 기본적으로 지원합니다

In [4]:
# 소켓 커넥션을 통해 수신된 스트림 데이터를 통해 데이터 프레임을 생성하는 예제입니다
socketReader = (
    spark
    .readStream
    .format("socket")
    .option("host", "localhost")
    .option("port", 9999)
    .load()
)

<br>

## 2. 워드 카운트 변환 로직 생성

### Step 2: 데이터 가공하기 - { counter }

> 이제부터는 라인을 단어로 구분하거나, 횟수를 세는 등의 일반적인 데이터프레임 연산을 수행합니다

* split 함수를 통해 생성된 array 를 Row 로 변환하기 위해 explode 함수를 사용합니다
* 공백을 " "으로 명시해도 되지만, \t, space 등을 정규식으로 인식하도록 \s 가 더 유용합니다

In [5]:
socketCounter = (
    socketReader
    .select(
        explode(
            split(col("value"), "\s")
        ).alias("word")
    )
    .groupBy("word")
    .count().alias("count")
)

> 여기까지는 실행되는 코드라기 보다는 어떻게 변환 할 지에 대한 가공에 대한 처리를 정의하는 과정이므로, show 함수 수행 시에는 스트림 데이터가 시작되지 않아 오류가 발생합니다

#### Stateless transformations
  - select, filter, map 등의 함수는 이전에 수행 되었던 로우가 다음 수행될 로우에 대해 영향이 없는 변환 작업을 스테이트리스 변환작업이라고 합니다
  - 즉, 이전 변환작업의 상태(state)가 이후 작업에 필요 없는(less) 변환(transformation) 작업입니다

#### Stateful transformations
  - 집계가 필요한 count 와 같은 함수는 이전의 누적된 카운트 상태가 필요한 변환작업을 스테이트풀 변환작업이라고 합니다
  - join, group, aggregating 과 연관된 모든 변환은 이에 해당하며, 이전 변환작업의 상태(state)가 필요한(ful) 변환 (transformation) 작업입니다
  - 대부분의 작업은 지원하지만 일부는 리소스 혹은 비용적인 면에서 수행되기 어려운 경우도 있습니다

<br>

## 3. 콘솔 출력 생성

### Step 3: 출력 싱크 테이블 및 모드 선택하기 - { writer }
> 여기까지 데이터 변환에 대한 정의를 했다면, 변환된 데이터 저장을 위한 싱크를 작성해야 합니다. 

* DataFrame.write 대신 DataFrame.writeStream 함수를 통해 DataStreamWriter 생성을 통해 생성된 데이터프레임을 저장합니다

In [6]:
# 아래와 같이 괄호()를 통해 묶는 경우 줄바꿈을 하더라도 backslash(\)를 통해 구분하지 않아도 되어 코드 가독성이 높아집니다
socketWriter = (
    socketCounter
    .writeStream
    .format("console") # 결과를 콘솔에 출력합니다
    .outputMode("complete") # 매번 전체 데이터를 내보냅니다
)

##### 출력 모드 별 설계 방향
* Append mode
  - 기본 (default) 출력 모드이며, 이전 데이터가 이후의 데이터에 영향을 주지 않도록 변환처리를 설계해야만 합니다.
  - 데이터의 추가만 가능한 모드이므로, 상태가 없는 스테이트리스 한 변환을 그대로 출력하는 예제가 대표적입니다
  - 집계를 수행하는 경우, 집계에 대한 킷값이 중복되어 출력될 것이므로, 시간에 대한 추가적인 필드를 저장하건, 단순 집계를 통해 처리는 가능합니다
    - Window 즉 Buffering 을 하지 않는 상황에서는 초 단위의 집계결과만 출력할 수 밖에 없기 때문에 이러한 성질을 이해하고 있어야 합니다
* Complete mode
  - 항상 모든 출력을 그대로 내보내는 모드이기 때문에, 전체 집계 데이터가 메모리에 남아 있어도 될만큼 충분히 작도록 변환 처리를 설계해야 합니다
  - 대부분 입력 데이터를 특정 집계 함수를 통해 최종 결과 지표들만 출력하는 경우가 많습니다
    - 위의 단어 숫자를 세는 예제와 같이 입력 데이터 전체가 아니라 상위 N개의 단어만 출력하게 됩니다
* Update mode
  - 마지막 트리거 이후에 발생한 결과를 갱신하는 모드이므로 출력 저장소가 업데이트를 지원해야만 합니다

##### 파일, 카프카 및 임의의 저장소 출력
> 기본적으로 파일이나 카프카와 같이 스트리밍 저장을 지원하는 싱크도 있지만 natively 지원하지 않는 다양한 오픈소스 저장소의 경우 (batch 에서는 저장이 가능한) foreachBatch 혹은 foreach API 를 통해 저장이 가능합니다. 다만, 이러한 경우 exactly-once 를 보장하지 못합니다|

<br>

## 4. 트리거 생성

### Step 4: 데이터 처리 상세 설계 - { trigger }
> 스트리밍 질의를 시작하기 전에 데이터 처리 실행을 위한 상세한 설계를 하는데, 얼마나 자주 혹은 어디에 저장할지 등을 결정합니다.


In [7]:
# 성공적으로 수행한 마지막 상태(state)를 저장하기 위한 지속성(persistent)있는 저장소(예: hdfs)의 위치를 지정하며, 실패한 작업의 재개 혹은 복구 시에 활용되어 exactly-once 보장.
queryName = "socket_word_count"
socketCheckpoint=f"{work_dir}/tmp/{queryName}"
socketTrigger = (
    socketWriter
    .queryName(queryName)
    .trigger(processingTime="1 second")
    .option("checkpointLocation", socketCheckpoint)
)

#### 트리거링 동작 방식에 대한 이해

* Default
  - 언제 기동될 지를 명시적으로 기술하지 않는 설정이 디폴트이며, **이전에 수행된 마이크로배치 작업이 종료된 직후**에 수행됩니다
* Processing time with trigger interval
  - 명시적으로 *ProcessingTime* 트리거를 *interval*과 함께 지정하여 고정된 간격으로 마이크로배치 작업이 수행됩니다
* Once
  - 단 한 번의 마이크로배치 스트리밍 쿼리가 수행되며, 신규로 수신된 모든 데이터를 한 번의 배치작업에서 수행됩니다.
    - 외부 스케줄러 시스템을 통해서 커스텀 스케줄을 통한 수행을 해야할 필요가 있는 경우 유용합니다
* Continuous
  - Spark 3.0 버전의 실험적이 모드이며 스트리밍 쿼리가 마이크로배치가 아니라 지속적인 데이터 처리를 수행합니다
    - 기존 마이크로배치가 초 단위의 레이턴시를 제공했다면 Continuous 는 밀리초 수준의 레이턴시를 제공합니다

<br>

#### 애플리케이션 기동 후에, 소켓 서버를 기동하여 터미널에서 키워드를 입력합니다
> 예제를 수행하고, 해당 스트리밍 데이터 소스로부터 읽고, 처리하고, 최종 출력까지 지속적으로 수행하게 됩니다. 노트북 특성상 표준 입출력이 노트북 컨테이너의 표준출력으로 나오기 때문에 노트북 컨테이너의 로그를 통해 확인이 가능합니다

* 표준 입력으로 문자열을 입력할 소켓 서버를 생성합니다
  - 해당 도커 컨테이너에 외부 포트를 publish 하지 않았기 때문에 컨테이너에 접속하여 넷캣(nc) 도구를 이용합니다
  - nc 는 터미널 환경에서 간단히 소켓 서버를 기동할 수 있는 커맨드라인 도구입니다
| 옵션 | 설명 |
| --- | --- |
| -l (Listen mode) | 연결을 할 수 있게 계속 요청을 듣고 있게 해주는 옵션 |
| -v (Verbose) | 상세 정보를 보여주는 옵션 |
| -p (port) | Listening 포트 |
<br>

* 아래와 같이 별도의 터미널을 통해 넷캣(nc) 명령어를 실행합니다
```bash
# (base) root@notebook: nc -lvp 9999 -n
Listening on 0.0.0.0 9999
Connection received on localhost 37600
hello world
data engineering intermediate course
```
<br>

* 표준 출력으로 스파크 스트리밍 실행 결과를 확인 합니다
```bash
$ docker compose logs -f notebook
21/06/27 04:27:23 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 7385 milliseconds
notebook  | -------------------------------------------
notebook  | Batch: 3
notebook  | -------------------------------------------
notebook  | +------+-----+
notebook  | |  word|count|
notebook  | +------+-----+
notebook  | |korean|    1|
notebook  | |   one|    1|
notebook  | | korea|    1|
notebook  | | hello|    1|
notebook  | |    is|    1|
notebook  | |     i|    1|
notebook  | | world|    1|
notebook  | |    am|    1|
notebook  | |number|    1|
notebook  | +------+-----+
```

<br>

## 5. 애플리케이션 기동 및 소켓 출력

### Step 5: 질의문 시작하기 - { query }

> 여태까지의 모든 작업을 수행하는 단계이며 최종 trigger 객체를 start 합니다

* 시작과 동시에 checkpointDir 로 지정한 경로가 생성됩니다
* start 함수는 nonblocking 메소드이므로 즉각적으로 query 객체를 반환합니다
  - 지속적인 결과를 확인하기 위해서는 해당 query.awaitTermination() 을 통해 대기합니다
  - timeout 값을 통해 타임아웃(초) 시간을 정할 수 있으며, 지정하지 않으면 Ctrl+C(SIGINT=2)까지 수행합니다
* 이번 예제에서는 터미널을 통해 interactive 한 수행을 위해 termination 하지 않습니다

In [8]:
socketQuery = socketTrigger.start()

In [9]:
socketQuery.stop()

### <font color=blue>1. [중급]</font> 소켓 서버를 통해 아래의 조건에 맞추어 스트리밍 애플리케이션을 생성 및 실행합니다

> 2교시에 학습한 전체 내역을 하나의 Cell 에 모두 포함하여 기동할 수 있도록 애플리케이션을 코딩해 주세요

* 소켓 소스
  - 호스트 : localhost
  - 포트 : 9999
* 변환 작업
  - 소켓으로 전달 받은 공백으로 구분된 문자열을 단어로 쪼개어(split, explode), "Word", "Count" 컬럼으로 alias 합니다
* 콘솔 싱크
  - 쿼리 : wordCount
  - 포맷 : console
  - 모드 : complete
* 트리거링
  - 1초에 한 번 트리거링
  - 체크포인트 : /home/jovyan/work/lgde-spark-stream/tmp/wordCount
* 애플리케이션
  - 타임아웃 : 1~3분 내외 (소켓 서버 테스트 할 시간)
  - 테스트 후 애플리케이션을 종료해 주세요

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

# step1: 데이터를 읽어올 스트림 리더를 생성합니다
wordCountHost = "localhost"
wordCountPort = 9999
wordCountReader = (
    spark
    .readStream
    .format("socket")
    .option("host", wordCountHost)
    .option("port", wordCountPort)
    .load()
)

# step2: 데이터로부터 단어의 수를 세는 카운터를 생성합니다
wordCounter = wordCountReader.select(explode(split(col("value"), "\s")).alias("word")).groupBy("Word").count().alias("Count")

# step3: 생성된 수치를 콘솔에 출력하는 출력을 생성합니다
queryName = "wordCount"
wordCountWriter = (
    wordCounter
    .writeStream
    .queryName(queryName) # 쿼리 테이블의 이름을 지정합니다
    .format("console") # 결과를 콘솔에 출력합니다
    .outputMode("complete") # 매번 전체 데이터를 내보냅니다
)

# step4: 얼마나 자주 수행될 지를 결정하는 트리거를 생성합니다
wordCountCheckpointDir = f"{work_dir}/tmp/{queryName}"
!rm -rf $wordCountCheckpointDir # 경우에 따라서 이미 존재하는 경로의 경우 오류가 발생할 수 있기 때문에 항상 제거합니다
wordCountTrigger = (
    wordCountWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", wordCountCheckpointDir)
)

# step5: 해당 
wordCountQuery = wordCountTrigger.start()
wordCountQuery.awaitTermination(60) # 노트북 특성상, 대기하면 다음 실행을 할 수 없으므로 1분간만 수행하고 종료합니다
wordCountQuery.stop() # 이후에 발생하는 처리는 수행되지 않으며 소켓 서버도 같이 종료됩니다
```

</details>

In [ ]:
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


<br>

## 6. 스파크 스트리밍 실행과정 이해

### Under the Hood of an Active Streaming Query

![figure.8-5](images/figure.8-5.png)

> 한 번 스파크 스트리밍 애플리케이션이 기동되면 아래의 내부적인 과정을 통해 작업이 수행됩니다. 여기서 반드시 기억해야 할 사항은 **'구조화된 스트리밍'은 Spark SQL 을 이용하여 수행되며, 'Spark SQL'의 최적화된 실행 엔진**을 통해 스트리밍 처리의 처리량과 레이턴시를 확보할 수 있습니다.

<br>

### 6.1 스파크 스트리밍 실행 과정

#### 1. 배치 작업과 마찬가지로 Logical Plan 을 분석 및 최적화 합니다

#### 2. 백그라운드 스레드를 통해 아래의 단계를 반복 합니다
* 2-1. 트리거링 간격에 맞게 데이터 소스에 새로운 데이터가 존재하는 지 확인합니다
* 2-2. 처리해야 할 데이터가 존재한다면 '마이크로 배치' 작업을 통해 소스로 부터 새로운 데이터를 계산한 결과를 출력 모드에 따라 싱크에 저장합니다
* 2-3. 매 '마이크로 배치' 마다, 정확한 범위의 데이터가 처리되며, 결정론적(deterministically)으로 재수행 되어도 완전히 동일한 범위를 보장합니다

#### 3. 쿼리가 종료되기 까지 아래와 같은 경우를 제외하면 반복은 지속됩니다
* 3-1. 쿼리 내부 오류에 의한 애플리케이션 실패 (혹은 클러스터의 장애)
* 3-2.애플리케이션 내에서 query.stop() 메소드의 명시적인 호출에 의한 종료
* 3-3.트리거가 Once 로 설정되어, 한 번의 '마이크로 배치' 작업이 종료된 경우


<br>

### 6.2 `Exactly-Once` 시맨틱을 보장하는 장애복구

> 한 번 실패한 '스파크 스트리밍' 작업을 재개하기 위해서는 'Spark Session'에서부터 마지막에 수행했던 과정을 그대로 수행되어야 하는데, 마지막으로 성공적으로 저장된 체크포인트 정보를 통해서, 이어서 수행될 수 있습니다.

* 반드시 동일한 '체크포인트' 경로를 지정해야 합니다
  - 해당 디렉토리는 유니크 아이덴티티 정보를 통해 쿼리의 생명주기를 결정합니다
  - 대상 디렉토리가 존재하지 않는다면, 처음부터 다시 수행되어야만 합니다
  - 체크포인트를 통해 마지막으로 성공한 바로 다음 '마이크로 배치'작업을 수행하며, 이전에 시도했던 범위와 완전히 동일한 범위의 작업을 시도하게 됩니다.


<br>

### 6.3 `End-to-End Exactly-Once`를 보장을 위한 요건 3가지
> '구조화된 스트리밍'은 아래의 몇가지 조건을 만족해야만 exactly-once 을 보장합니다

#### 하나. 재수행 가능한 스트리밍 소스 - Replayable streaming sources
* 과거에 실패했던 작업의 위치에서 동일한 작업의 범위의 데이터를 다시 읽어올 수 있을 것
  - 카프카의 경우 데이터가 삭제되지만 않았다면, 오프셋 정보를 통해서 특정 시점 이후의 데이터를 가져올 수 있습니다
  - 예제의 소켓 버서의 경우 한 번 수신된 데이터를 다시 수신할 수 없기 때문에 exactly-once 를 보장하지 못합니다

#### 둘. 결정론적인 계산 - Deterministic computations
* 입력 조건이 동일한 경우, 항상 결과가 동일한 연산이 수행될 수 있는 변환작업일 것
  - 동일한 키가 발생할 수 있는 컬럼을 이용하는 정렬의 경우 여러번 실행하는 경우 결과가 흔들릴 수 있습니다 (분산환경임을 유의할 것)

#### 셋. 멱등한 스트리밍 싱크 - Idempotent streaming sink
* '마이크로 배치'작업이 여러번 수행 되더라도, 항상 동일한 결과를 가질 수 있을 것
  - 반복 수행을 통해 일부의 데이터만 반복 수행되더라도, 항상 Upsert 하거나, Truncate 혹은 Range Delete 후, Insert 하도록 구현하면 멱등성을 보장할 수 있습니다


<br>

### 6.4 호환성을 보장하는 *감내할 수 있는 변경* 수준 3가지
> '구조화된 스트리밍'의 경우 아래와 같이 체크포인트 상태에 영향을 주지 않는 수준의 마이너한 수정후 실행이 가능합니다 

#### 하나. 특정 라인을 필터 하는 함수 적용
* 문제가 발생할 가능성이 있는 로우를 제거하는 예제
```python
spark> filteredLines = lines.filter("isCorrupted(value) = false")
spark> words = filteredLines.select(split(col("value"), "\s").alias("word"))
```

#### 둘. 소스와 싱크 옵션의 변경
* `host, port 옵션은 변경되어서는 안`되지만, 출력 로우의 수 등의 옵션은 변경할 수 있습니다
```python
spark> writerStream.format("console").option("numRows", "100")
```

#### 셋. 데이터 처리 상세 옵션
* `체크포인트 위치는 변경되어서는 안`되지만, 트리거링 간격과 같은 옵션들은 변경할 수 있습니다
